In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import norm
from statsmodels.stats.proportion import confint_proportions_2indep
from statsmodels.stats.proportion import proportions_ztest
# import the beta function from scipy.special
from scipy.special import beta as beta_function
from scipy.stats import beta as beta_dist


# A/B Test Methodologies: Null Hypothesis Significance Testing vs. Bayesian Approaches

When evaluating new user experiences (UX) — such as launching **passkeys** and measuring their impact on abandonment rates — we need a way to decide whether a new design is better, worse, or equivalent to the current one.  
This notebook compares two major approaches:

- **Null Hypothesis Significance Testing (NHST)** — the long-standing statistical framework, widely used but often conceptually tricky and difficult to interpret in practical decision-making.
- **Bayesian methods** — increasingly popular because they offer more flexibility and produce results that are often easier to interpret directly when deciding actions (e.g., when to shift more traffic from a control to a new variant).

---

## Test Setup: Control Group vs. Variants

We assume an existing digital identity creation flow with a **completion rate of ~20%** (meaning ~80% of users abandon).  
Our test design:

- Keep **90%** of traffic on the current experience as the **control group**.
- Send the remaining traffic to one or more **variants** $A, B, C$.

The test will be conducted in 2 steps, first we need to determine that each new experience is **no worse** than the current one, accepting small degradaition as unavoidable as we are adding more pages and clicks  — then after establishing we haven't degraded the experience, shifting more traffic to the variants and decidng which is the better-performing variant.

The type of A/B test — where the first goal is to ensure a new design does **not degrade** the experience — is called a **non-inferiority test** (explained below).


## Null Hypothesis Significance Testing (NHST) Methodology

At a high level the methodology is : assume what you don't want to see or have, call it the null hypothesis, for instance, the drug has no effect or in our case the new experience significantly increase abandonment, run the test, look at the result if and if the result looks  unlikely enough, as if low enough probabilityy to happen (e.g. like that 5% to see what we observe) under the Null Hypothesis, then reject the hypothesis. Note 2 thing immediately, you havent really said whether the opposite hypothesis is true, just that your reject the null. Also "unlikely enough" is defined by a somewhat arbitrary number (5% chance).

The method boils down to compute the probabilityy of the observation given the hypothesis, as we will see later the Bayesian approach is going the other way, it computes the probabilty of the hypothesis given the observation, which is a totally different metric

### Framing of the problem using Random Variables

This abandonment rate of a  UX is usually modelled as 2 Bernoulli Random Variables $X_C$ and $X_A$ from which we will draw repeatedly. Bernoulli just mean we present a choice with onlyy 2 possible outcome success/failure, convert/abandon, up/down etc.. presenting the new CX repeatedly to a bunch of end users.

$X_C$ being the UX and Bernoulli RV presented to control control group  $X_A$ being the variant A presented. They are both assumed to have for codomain $\mathcal{X}_C=\mathcal{X}_A=\{0,1\}$ that is a boolean i.e. it converts or does not convert. Here "convert" means the user perform the action they intended to do vs. abandoning the journey, e.g. create a passkey. There a possibility with a techincal failure but this would not be their choice so it would be counted as a success for the purpose of this AB test.

The way NHST usually model the difference between a control and variant convertion rate is to define a new random variables called the "sample proportions" which is the sum of n instances of the Bernoulli ones defined above, divided by n. For this "sample proportion" Let's using the notation  $\hat{p}_C$ and $\hat{p}_A$ for the control and the variant respectively , defined as

 $ \hat{p_C}=\frac{1}{n}\sum_{i=1}^n X_{Ci}$ and 
 $ \hat{p_A}=\frac{1}{n}\sum_{i=1}^n X_{Ai}$
 
 each sample proportion can be interprested as 2 things :

* $ \hat{p_C}$ and  $ \hat{p_A}$ can be interpreted as an ordinary random variables with codomain $\{0,\frac{1}{n},\frac{2}{n},...\frac{n}{n}\}$
* $\hat{p_C}$ and  $ \hat{p_A}$ can also be interpreted as estimators (in the technical sense of the word "estimator" used in classical statistics) of the expected value $E(p_C)$ and $E(p_A)$. In general regardless of whether it is a control or a variant $E(\hat{p}) \rightarrow E(p)$ because of the law of large numbers applied to a Bernoulli/Binomial.

The notation with a "little hat" over the letter is the convention in statistics to denote estimators.

An estimator is a function from the n-cartesian product of the space of realizations of X (on itself n times)  to the domain of the parameter of interest, if we  use the notation codomain(X) = $\mathcal{X}$, it is then a mapping from $\mathcal{X}^n$ to some real value for a parameter of original random variable X, in this case $\hat{p}_A: \mathcal{X}^n \rightarrow \{0,\frac{1}{n},\frac{2}{n},...,\frac{n}{n}\}$ Because it it is a sum of Bernoullis, the estimator RV follows binomial distribution centered on the expected value. It is always true that a discrete binomial distribution can be approximated by a continuous gaussian as n grows (clipping the tails)


#### Variance and standard devation of a sum of Bernoulli or Binomial

in all the rest of the discussion we are going to use a lot of the variance and standard deviation of the sample proportion as it it used to defined what is called the "standard error" under various hypothesis. This is just a reminder of how they are computed

Note that for a "single" Bernoulli random variable $X$ we have $Var(X) = p (1-p)$ and therefore 
  
$Var(\frac{1}{n}\sum_{i=1}^n X_i) = \frac{1}{n^2}Var(\sum_{i=1}^n X_i)$

$Var(\frac{1}{n}\sum_{i=1}^n X_i) = \frac{1}{n^2}n(p(1-p))$


$Var(\frac{1}{n}\sum_{i=1}^n X_i) = \frac{1}{n}(p(1-p))$

so $\boxed{Var(\hat{p})= \frac{p(1-p)}{n}}$



###  Notation for the  Estimator of the difference in proportions

To make the rest of the discussion easier to read  we are going to define the difference in estimator $\hat{p_A}$ and $\hat{p_C}$ as $\hat\Delta = \hat{p_A} - \hat{p_C}$ which is also an estimator of the expected value of random varaible $\Delta = p_A - p_C$


### Null Hypothesis (H₀):


As we said earlier the  "effect" we are trying to avoid is if the new passkey creation pages would degrade the overall UX  so much that it may impacts our other business metrics. In the classical framing of a non-inferiority test, the  Null Hypothesis is the "bad thing" we are trying to reject in our case it would mean : "some degradation".

"some degradation" can be described with a simple inequality like  $E(p_A) \le E(p_C) - \epsilon $ where $\epsilon$ is a tiny degradation we would find acceptable for instance 0.03 (3%).

We will rewrite

$H_0$ as $\boxed{H_0 = E(\Delta) \leq -\epsilon}$




### Alternative Hypothesis (H₁):
 The conversion rate for the variant is higher than $\boxed{E(\Delta) > -\epsilon}$



### Boundary Hypothesis

Below we will make use of the hypothesis that the difference in conversion is exactly the small acceptable one  that is

$\boxed{E(\Delta) = -\epsilon}$


#### Numerical example 

Each realization is given after each experience by the folowing counts

$n_C$: Number of visitors in the control group

$x_C$ : Number of conversions observed (realization) in a given control group

$n_A$: Number of visitors in the variant group

$x_A$ : Number of conversions observed (realization) in the variant group

$\hat{\Delta_{obs}}$ the observed value of the difference between the proportions

$\epsilon$ : acceptable degradation in the difference between the proportions



In [ ]:
control_group_conversion_rate = 0.2 # based on historical data
nC = 7000
xC_observed = nC * control_group_conversion_rate
nA = 150
xA_observed = 33
epsilon = 0.03
alpha = 0.05
hatpC_observed = xC_observed / nC
hatpA_observed = xA_observed  / nA
hatDelta_observed = hatpA_observed - hatpC_observed

print(f"Realization of difference in conversion rate estimator: {hatDelta_observed:.4f}")
print(f"Control group realization of conversion rate estimator: {hatpC_observed:.4f}")
print(f"Treatment group realization of conversion rate estimator: {hatpA_observed:.4f}")

### Standard Deviation of the estimator $\hat{\Delta}$ a.k.a Standard Error in frequentist/classic statistics.

The NHST methodolgy involve computing first an estimatation of the standard deviation for the estimator $\hat{\Delta}$, then to see how likely it then comparing it with the actual values we get from the difference in proportion in the experiment, then based on this decide if the difference observed is far enough (or close enough) to the theoritical value to reject or not reject the hypothesis based on some accpeted risks in the deicison rules (power, confidence internval)

THis is the first problem NHST has to tackle we have no idea what the true standar deviation is, so all varation on NHST use a "hack" they call the plug in principle, which basically use the current experiement results , to "plug in" some formula to esimate the unknown standard deviation. But note the circularity of the method:

1. We want to test if the data is unusual under H₀
2. To measure "unusual," we need the standard error under H₀
3. But SE depends on the unknown θ, so we plug in p̂ (the data itself!)
4. We then use this data-derived SE to judge whether the data is unusual

It's like: *Let me use my single measurement to tell me how variable my measurements are, then use that to decide if my measurement is surprising*

NHST proponent justify it because if ou repeateded the same experiment many times, at infinity would converge to the true value "on average".But you onlyy have ONE sample!
You don't know if your particular p̂ is:
Close to the true value (plug-in works well)
An outlier (plug-in is terrible)

Why Frequentists live with it:

* Long-run frequency interpretation: "If we repeat this procedure many times, it has correct coverage"
* Pragmatism: They need something computable
* Simulation evidence: It works "reasonably well" in practice for moderate n, but "reasonably well" can be challeng see goole "the crisis of replication" to get a sense of the numerous failures.
* No alternative: In the frequentist framework, parameters are fixed unknowns, not random variables

### The various plugin Hacks


#### If hypotehsis is "no effect" the Wald pooling method:

If instead of "non inferiority" the null hypothesis  is "no effect" ( we will have that later for H1), the plug in hack is to record the observed conversion rates (estimators) for each proportion then  combine (pool) as under this very specific hypothesis they have the exact same expected value so a larger sample created through pooling is a better estimator of the true value, whatever it is.

the realization of $\hat{p}_A$  is $(\hat{p}_A(\omega) = \frac{x_A}{n_A})$

the associated realization of $\hat{p}_C$ is  $(\hat{p}_C(\omega) = \frac{x_C}{n_C})$

Then there is a theoritical estimator for the combine exeprience IF the null hypothesis was holding and there was not difference A and the control there would be a random variable for the estimator of both being pooled

$\hat{p}(\omega_{pool}) = \frac{x_A + x_V}{n_A + n_C}$

Then we try to compute the variance standard deviation of the difference between the conversion rate of the control vs. the variance. The variance of the difference beteww the 2 proportion is the sum of their variance, so

$Var(\hat{p}_A - \hat{p}_C ) = Var(\hat{p}_C) + Var(\hat{p}_A)$

Also because both $\hat{p_C}$ and $\hat{p_A}$ are Bernoulli RV, and using this "plug in " principal; we compute the sum of variance of the estimators also using the pooled sample reading as:

$Var(\hat{p}_A - \hat{p}_C ) = \frac{\hat{p}(\omega_{pool})(1- \hat{p}(\omega_{pool}))}{n_A} + \frac{\hat{p}(\omega_{pool})(1- \hat{p}(\omega_{pool}))}{n_C} = \hat{p}(\omega_{pool})(1- \hat{p}(\omega_{pool})) (\frac{1}{n_C} + \frac{1}{n_A})$

and the standard error which is just the standard deviation of the same quantity  also using the plug in principl the standard devation of the estimators that statisticain call "standard error" is

$\boxed{WaldPooled SE = \sqrt{\hat{p}(\omega_{pool})(1- \hat{p}(\omega_{pool})) (\frac{1}{n_C} + \frac{1}{n_A})}}$

this is sometimnes called "pooled standard error" or "standard error of the difference between two independent proportions"

From this we conmpute the z-score whuich is a dimenionlyess value counting the number of standard deviation that the difference observed is from the theoritical difference which shoiuld be zero under the null hypothesis (approximated by the pooled one, sktechy). So a simple ivisino of the observed devation over teh standard deviation of hte diference

$z= \frac{\frac{x_A}{n_A} - \frac{x_C}{n_C}}{SE}$



#### If hyopthesis is "non inferiority", use the Wald Unpooled Standard Error:

If the hypotheis is not "no effect", but "on inferioity" we cannot assume that each sample is drawn from the same random variable, we actually explicitely say it is not because of the epsilon among other thing. So the first quick and dirt approch is just to sum the separate variances (see above variance of Bernoulli) as variances are additive when random variable are added  or substracted in our case, and are assymed to be independent, then take the square roo t pf that sum to get to a standard deviation:

$\hat{WaldUnpooledSE} = \sqrt{\frac{\hat{p_A(w)}(1-\hat{p_A(w)})}{n_A} + \frac{\hat{p_C(w)}(1-\hat{p_C(w)})}{n_C}}$

Note that we should be using hte expected value of $E(p_A)$ and $E(p_B)$ in the formula but we don't know them, so we use the so call "plug in" trick and replace them but whatever values we get. That is why this is a quick and dirty approach that can produce questionable results depending on the specific phenomenon and sizes of samples.

#### Slightly Better for "non inferiority"  Newcombe (score-based / Wilson)

Much better actual coverage than Wald, especially with imbalanced samples or extreme p.
Still fairly simple to compute (closed-form formulas exist, or can be coded).

we wont' cover it here but it is still a plug-in hacks suffering from circularity argument and n=1 problem.

#### Miettinen–Nurminen

Widely used in clinical trials and non-inferiority contexts and in some regulated industry like pharmaceitical (recommended by the FDA)

Very complex and hard to explain, and is still fundmentally a plug in hack with the same circular reasoning and  n=1 We wont cover it.


Numerical examples

In [ ]:
pooled_proportion = (xC_observed + xA_observed) / (nC + nA)
wald_pooled_SE = (pooled_proportion * (1 - pooled_proportion) * (1/nC + 1/nA))**0.5
print(f"Wald Pooled Standard Error: {wald_pooled_SE:.4f}")
wald_unpooled_SE = ((hatpC_observed * (1 - hatpC_observed) / nC) + (hatpA_observed * (1 - hatpA_observed) / nA))**0.5
print(f"Wald Unpooled Standard Error: {wald_unpooled_SE:.4f}")

#### Probability of False Postive, pvalue, signifiance level $\alpha$ (a.k.a confidence by some people) and critical value


Once we approximated the  standard error SE (which is the standard devation of $\hat{\Delta}$) as explained above, teh NHST methodology then also fixed the expected value of $\hat{\Delta}$ under H0. The idea is that if you fix bothe the expected value and standar deviation , you can then work with a binomial or guassian probabilityy distribution to compute whatever event probabilityy you want. Even though the process is descrete and we should use a binomial continyusous gaussian are used in practice as htey are easier to manipulate in equations and are good approxmiation of binomial.




The other simplificaiton is that although H0 is an inquality $E(\Delta) \leq -\epsilon$ we are going to actuall use $E(\Delta) = -\epsilon$ to get a single probabilityy distribution to work with, with the rationale that the boundary is the least favorable case, that is if we reject H0 at the boundary $-\epsilon$ logically we would reject it also at anythng with $E(\Delta) \leq -\epsilon$, as our observation would be get an even lower probabilityy if the mean shifted  to the left.So using the boundary value epsilon is that one that makes it the most difficult to cross the alpha threshold and the most difficult to reject $H_0$

So with all that in place we can finally model H0 with a gaussian of mean $\mu = -\epsilon$ and standard deviation $\sigma = \hat{SE}$ 

$N(\mu, \sigma)$

Now we can compute the p-value which is the probabilityy of observing or something like the sample we got or more extreme in the direction of H1. In our case it would be probabilityy of the sample being in $[\hat{\Delta_{obs}}, +\infty]$ which can be comnputed by integrating the right tail of the gaussian from $\hat{\Delta_{obs}}$ to $+\infty$. This integral of the right tail is called the survival function of the gaussian and can be computed direct in python.




The direct way of computing the survival function and therefore the pvalue  directly from the observation $\hat{\Delta_{obs}}$ would be to integrate

pvalue $ = \int_{ \hat{\Delta_{obs}} }^{+\infty} \frac{1}{\sqrt{2 \pi}(SE)} e^{-\frac{(x - (-\epsilon))^2}{2(SE)^2}} dx$

Using a common notation where $\Phi$ is the cumulative distribtiuon of the standard normal N(0,1), the survival would altneratively expressed as this if one does not have something that compute survial function directly

pvalue = $1 - \Phi(\frac{\hat{\Delta_{obs}} - \mu}{\sigma})$

that we will compare to the $\alpha$, in our case 0.05, if the the probabilityy of seeing what we observed or something even more favorable (the right tail) is 5% or less, we reject the null hypothesis H0 because we are seeing is very unlikely under H0. The critical value is the inverse of the right tail integratiojn, is the the value of hte observation that would make the probability of seeing the sample 5% or lower in our setup. The formula to inverse it from the $\alpha$ is

$c = \mu  + \sigma \Phi^{-1}(1 - \alpha)$

Below is a full computation and a vizualization of the parameters along the right tail



In [ ]:
SE_H0 = wald_unpooled_SE
from scipy.stats import norm

mu_H0 = -epsilon    # mean
sigma_HO = SE_H0  # standard deviation
x = hatDelta_observed  # value to evaluate

# Survival function P(X > x)
p_value = norm.sf(x, loc=mu_H0, scale=sigma_HO)
print(f'p-value (one-sided): {p_value:.4f}')

critical_value = norm.isf(alpha, loc=mu_H0, scale=sigma_HO)
print(f"Critical value for p-value={alpha:.4f}: {critical_value:.4f}")



In [ ]:
# Plot the Gaussian N(mu, sigma) and shade the right-tail area beyond x

# Use previously defined values; recompute to be robust
SE_H0 = wald_unpooled_SE
mu_H0 = -epsilon
sigma_HO = SE_H0
x0 = hatDelta_observed

# Domain for plotting (±6σ around the mean, clipped to reasonable bounds)
left = mu_H0 - 6 * sigma_HO
right = mu_H0 + 6 * sigma_HO
xs = np.linspace(left, right, 1000)
pdf = norm.pdf(xs, loc=mu_H0, scale=sigma_HO)

# Right-tail probabilityy
p = norm.sf(x0, loc=mu_H0, scale=sigma_HO)

# Critical value at significance alpha (one-sided)
crit_x = norm.ppf(1 - alpha, loc=mu_H0, scale=sigma_HO)

# Plot
fig, ax = plt.subplots(figsize=(7.5, 4.5))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.plot(xs, pdf, color="C0", lw=2, label="Density")

# Shade right tail
mask = xs >= x0
ax.fill_between(xs[mask], pdf[mask], color="C1", alpha=0.35, label=f"Right tail p = {p:.4g}")

# Vertical line at observed x
ax.axvline(x0, color="C1", ls="--", lw=1.5, label=f"observed delta = {x0:.4f}")

# Vertical line at critical value
ax.axvline(crit_x, color="C2", ls="-.", lw=1.5, label=f"critical value c = {crit_x:.4f}")

# Vertical line at the mean for the null hypothesis H0
ax.axvline(-epsilon, color="k", ls=":", lw=1.5, label=f"mean under H0 = {-epsilon:.4f}")

# Decorations
ax.set_title(f"Normal(μ={mu_H0:.4f}, σ={sigma_HO:.4f}) — Right-tail beyond x")
ax.set_xlabel("Δ (difference in proportions) under the null H0")
ax.set_ylabel("Probability density under H0")
ax.legend(loc="best")
ax.grid(True, ls=":", alpha=0.5)
plt.tight_layout()
# plt.show(), display handled by notebook



#### traditional presentation of the same conmputation using z-scores

The traditional ways for NHST to compute the p value is not to use the measurment $\hat{\Delta_{obs}}$ directly but to normalize it to a standard normal and centered on zero and on standard deviation 1 N(0,1). This mormalization is done by transforming the measurment into a so called z-score:

The the z-score is just the way you can rescale an arbitrary Gaussian to the Normal by substracting the meand and dividing by the standard deviation $Z = \frac{X-\mu}{\sigma}$

$Z_{NI} = \frac{\hat{\Delta} - (E(\Delta_{H_{boundary}}))}{SE}$ 

$= \frac{\hat{\Delta} - (-\epsilon)}{SE}$ 

$\boxed{Z_{NI}= \frac{\hat{\Delta} + \epsilon}{SE}}$

Then  $\frac{\hat{\Delta} + \epsilon}{SE}$ is a normal N(0,1) and 

$p(\hat{\Delta} \ge \hat{\Delta(w)})$

$ = p(\frac{\hat{\Delta} + \epsilon}{SE} \ge \frac{\hat{\Delta(w) +\epsilon}}{SE })$

Then we can compute the survial function but this timne on a standard normal and from the z-score value

$\int_{ Z_{NI} }^{+\infty} \frac{1}{\sqrt{2 \pi}} e^{-\frac{1}{2}z^2} dz$

which give the exact same results:



In [ ]:
zni = (hatDelta_observed + epsilon) / SE_H0
p_zni = norm.sf(zni)
print(f"z_NI: {zni:.4f}, p-value: {p_zni:.4f}")

#### False positive
In this NHST the pvalue is also the probability of a false positive that is the probabilityy of rejecting the null hopythisis (saying htere is no degradation) while there is acxtually a degradation. So it is setup our risk of make the wrong decision is 5%

#### Probability of having false negative a.k.a type 2 error , Power and sample size required to get to signficance under those conditions

Conversely , "False negative" in the context of a non inferiority test is the probabilityy of failing to reject given the fact that H1 is true, meaning we fail to clear the non inferiority test, while the new UX is actually non inferior (i.e. as good or better than the old one). So we have the same issue as for the false poistive computation, we have to pick a value for what expected value for the $\Delta$ but this time under H1. As we cannot really work with a range of expecgted value for $\Delta$  we need a single value to fix the gaussian we are going to integrate. Here possible choice is to also pick a boundary which is $E(\Delta) = E(p_C)$ so whatever the control group mean was or is (depending on how much historical data we have). This is usally called "minimum effect size we want to detect" as is really a business decision, it is a bit arbitrary. Pickig our baseline is just one option

To model the estimator under the alternative hypothesis H1, we need to pick another expected value for the $\Delta$, since this is for H1 which means the same or better, we pick the bare minimum we need to make that statement that is "the same" and an expected value of zero for the Delta. Under these condition we can pool the samples to get an estimator of the variance and standard error as we are assuming they are distributed indentically

$SE | H1 = WaldPooled SE = \sqrt{\hat{p}(\omega_{pool})(1- \hat{p}(\omega_{pool})) (\frac{1}{n_C} + \frac{1}{n_A})}$

With that in place we compute the probability of the estimator coming up "higher" but with the same critical value as it was established ahead of time with the significance level alpha.

The Beta is the probability that we observed something lower and up to the critical value defined by inverting the p-value but under H1, meaning the distribution is centered on the H1 expected value, in our case it is zero.

This computation will give us whatever it gives us. If we want to TARGET a Power of 0.8 (which means β = 0.2, or 20% Type II error rate), we can compute the sample sizes (implied in SE) that will make us reach that level.



In [ ]:
SE_H1 = wald_pooled_SE
mu_H1 = 0
sigma_H1 = SE_H1
x = critical_value
beta = norm.cdf(x, loc=mu_H1, scale=sigma_H1)
print(f"probabilityy of false negative a.k.a β a.k.a type 2 errors,  at critical value : {beta:.4f}")
power = 1 - beta
print(f"Power (1 - β): {power:.4f}")


In [ ]:
# Domain for plotting (±6σ around the mean, clipped to reasonable bounds)
left = mu_H1 - 6 * sigma_H1
right = mu_H1 + 6 * sigma_H1
xs = np.linspace(left, right, 1000)
pdf = norm.pdf(xs, loc=mu_H1, scale=sigma_H1)

# Plot
fig, ax = plt.subplots(figsize=(7.5, 4.5))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.plot(xs, pdf, color="C0", lw=2, label="Density under H₁")

# Shade left tail (Type II error region)
mask = xs <= critical_value
ax.fill_between(xs[mask], pdf[mask], color="C1", alpha=0.35, 
                label=f"Type II error β = {beta:.4g}")

# Vertical line at critical value
ax.axvline(critical_value, color="C2", ls="-.", lw=1.5, 
           label=f"critical value c = {critical_value:.4f}")

# Vertical line at observed delta
ax.axvline(hatDelta_observed, color="C1", ls="--", lw=1.5, 
           label=f"observed delta = {hatDelta_observed:.4f}")

# Vertical line at the mean under H1
ax.axvline(mu_H1, color="k", ls=":", lw=1.5, 
           label=f"mean under H₁ = {mu_H1:.4f}")

# Vertical line at the H0 boundary (for reference)
ax.axvline(-epsilon, color="gray", ls=":", lw=1.5, alpha=0.7,
           label=f"H₀ boundary = {-epsilon:.4f}")

# Decorations
ax.set_title(f"Normal(μ={mu_H1:.4f}, σ={sigma_H1:.4f}) — Type II Error (β) and Power")
ax.set_xlabel("Δ (difference in proportions) under H₁")
ax.set_ylabel("Probability density under H₁")
ax.legend(loc="best", fontsize=9)
ax.grid(True, ls=":", alpha=0.5)

# Add text annotation for power
power_text = f"Power = 1 - β = {power:.4f}"
ax.text(0.98, 0.95, power_text, transform=ax.transAxes, 
        fontsize=10, verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()

### Bayesian Approach

In contrast to NHST the Bayesian approach is conceptually simpler, instead of artificially considering only 2 possible expected value boundaries for $\Delta$ one for H0 and one for H1, we assume we just don't know that $E(\Delta)$ can take any value between 0 and 1 , and this is just itself random variable but one that we can quantify accross all possible world

#### Using Beta distribution as a tool to model our 'pior belief"

For kind of Bernouilli trial we are dealin wiht (i.e. success/failure, convert/abandom etc...) the most convienent model for our prior belive is what is know as a Beta distribution, note that the anme "Beta" is unfortunate it as it sonds like the Beta in NHST, but it  has nothint ot with it, is is a probability distribution depending on 2 parameters which depening on how they are set can model "we know nothing a piorir" a.k.a an uniformative prior, or  "we don't know much but just that the coversion rate should be aropund 20% but we may be surprised" to  very strong prior based on tons of historical data "the conversion shold be ver close to 20%"

Note tha the Beta distribution is deined on the interval [0,1] as it is modeling a probability, so that'teh probability of a probability (meta probability)

Here are  few example of priors that coud be used, in cour case we will try with a totally unfornative priot (we know nothing) and weakly one (shoub be around 17% but we are not sure).

In [ ]:
x = np.linspace(0, 1, 1000)

# Four cases:
cases = [
    ("Uninformative (flat)", ("single", (1, 1))),
    ("Weakly informative (centered, high entropy)", ("single", (3, 12))),   # mean=0.2
    ("Strong conviction (centered, low entropy)", ("single", (200, 800))),  # mean=0.2
    ("Bi-modal (mixture of Betas)", ("mixture", ((5, 20, 0.5), (20, 5, 0.5))))
]

fig, axes = plt.subplots(2, 2, figsize=(10, 7), sharex=True, sharey=True)

for ax, (title, (kind, params)) in zip(axes.ravel(), cases):
    if kind == "single":
        a, b = params
        y = beta_dist.pdf(x, a, b)
        mean = a / (a + b)
        ci_low, ci_high = beta_dist.ppf([0.025, 0.975], a, b)

        ax.plot(x, y, color="C0", lw=2)
        ax.fill_between(x, y, color="C0", alpha=0.12)
        ax.axvline(mean, color="k", ls=":", lw=1.2, label=f"mean={mean:.3f}")
        ax.axvline(ci_low, color="C1", ls="--", lw=1, label="95% CI")
        ax.axvline(ci_high, color="C1", ls="--", lw=1)
        ax.set_title(f"{title}\nBeta(α={a}, β={b})", fontsize=10)
        ax.legend(fontsize=8, loc="best")
    else:
        # Mixture of two Betas: (a1, b1, w1), (a2, b2, w2)
        (a1, b1, w1), (a2, b2, w2) = params
        y = w1 * beta_dist.pdf(x, a1, b1) + w2 * beta_dist.pdf(x, a2, b2)
        m1, m2 = a1 / (a1 + b1), a2 / (a2 + b2)
        mix_mean = w1 * m1 + w2 * m2

        ax.plot(x, y, color="C0", lw=2, label="mixture pdf")
        ax.fill_between(x, y, color="C0", alpha=0.12)
        # Component means
        ax.axvline(m1, color="C2", ls="--", lw=1, label=f"mean₁={m1:.2f}")
        ax.axvline(m2, color="C3", ls="--", lw=1, label=f"mean₂={m2:.2f}")
        # Mixture mean
        ax.axvline(mix_mean, color="k", ls=":", lw=1.2, label=f"mixture mean={mix_mean:.2f}")
        ax.set_title(f"{title}\n{w1:.1f}·Beta({a1},{b1}) + {w2:.1f}·Beta({a2},{b2})", fontsize=10)
        ax.legend(fontsize=8, loc="best")

for ax in axes[-1]:
    ax.set_xlabel("p")
for ax in axes[:, 0]:
    ax.set_ylabel("density")

fig.suptitle("Four Beta Priors: flat, weakly centered, strongly centered, bi-modal", fontsize=12)
plt.tight_layout(rect=[0, 0, 1, 0.95])

#### Conservative approach assuming we know nothing (a.k.a non informative prior)

For the Variant A, assuming we start with an non informative prior meaning that we have no idea, p_A could range from 0 to 1 with equal probabilityy model by the beta function $Beta(1,1)$  
 
after n trials and k success the posterior probabilityy distribution due to the property of the Beta function is
 
$Beta(xA+1, n_A - xA+1)$ 

This derived through Bayes Theorem and how the Beta function can be integrated

The expected value for $E(Beta(\alpha,\beta)) = \frac{\alpha}{\alpha + \beta}$

which for a bayesian posterior becomes

$E(Beta(xA+1,nA−xA+1)) = \frac{xA+1}{xA+1 + nA−xA+1} = \frac{xA+1}{nA+2}$

In [ ]:
expected_value_posterior = (xA_observed + 1) / (nA + 2)
print(f"Expected value of posterior distribution for p_A: {expected_value_posterior:.4f}")

Now let's compute the credible intervals and visualize the prior and posterior distributions.

In [ ]:
# Posterior parameters
alpha = xA_observed + 1
beta_param = nA - xA_observed + 1

# Ensure we use the beta distribution from scipy.stats
from scipy.stats import beta as beta_dist

# Compute 95% credible interval (2.5th and 97.5th percentiles)
p_L = beta_dist.ppf(0.025, alpha, beta_param)
p_U = beta_dist.ppf(0.975, alpha, beta_param)

# Output the result
print(f"95% Credible Interval for p: [{p_L:.4f}, {p_U:.4f}]")

In [ ]:
# compute the 99% credible interval (1th and 99th percentiles)
p_L_99 = beta_dist.ppf(0.005, alpha, beta_param)
p_U_99 = beta_dist.ppf(0.995, alpha, beta_param)
# Output the result
print(f"99% Credible Interval for p: [{p_L_99:.4f}, {p_U_99:.4f}]")


In [ ]:
# Visualize non-informative prior vs posterior
x_range = np.linspace(0, 0.5, 1000)
prior_noninformative_pdf = beta_dist.pdf(x_range, 1, 1)  # Beta(1,1) - uniform
posterior_noninformative_pdf = beta_dist.pdf(x_range, alpha, beta_param)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_range, prior_noninformative_pdf, 'b--', lw=2, label='Prior: Beta(1, 1) - Non-informative')
ax.plot(x_range, posterior_noninformative_pdf, 'r-', lw=2, label=f'Posterior: Beta({alpha:.1f}, {beta_param:.1f})')

# Mark the non-inferiority boundary
ax.axvline(control_group_conversion_rate - epsilon, color='k', ls=':', lw=1.5, 
           label=f'Non-inferiority boundary = {control_group_conversion_rate - epsilon:.2f}')

# Mark the control conversion rate
ax.axvline(control_group_conversion_rate, color='g', ls=':', lw=1.5, 
           label=f'Control conversion rate = {control_group_conversion_rate:.2f}')

# Shade the 95% credible interval
mask = (x_range >= p_L) & (x_range <= p_U)
ax.fill_between(x_range[mask], posterior_noninformative_pdf[mask], alpha=0.3, color='red', 
                label='95% Credible Interval')

ax.set_xlabel('Conversion rate p_A', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title('Non-informative Prior vs Posterior Distribution', fontsize=14)
ax.legend(loc='best')
ax.grid(True, ls=':', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Compute probability that variant is non-inferior using non-informative prior
prob_non_inferior_noninformative = 1 - beta_dist.cdf(control_group_conversion_rate - epsilon, 
                                                       alpha, beta_param)
print(f"Probability that variant is non-inferior (non-informative prior): {prob_non_inferior_noninformative:.4f}")
print(f"This means there's a {prob_non_inferior_noninformative*100:.2f}% probability that the variant conversion rate is above {control_group_conversion_rate - epsilon:.2f}")

#### Weakly Informative prior approach using historical data

Instead of using a non-informative prior Beta(1,1), we can incorporate our historical knowledge. We know the control group has a conversion rate of 0.2, and we want to test for non-inferiority with margin epsilon = 0.03. 

For the variant, we'll use a prior centered at 0.2 - 0.03 = 0.17 (the boundary of non-inferiority), but with high entropy to reflect our uncertainty. A Beta distribution with mean μ and relatively small α and β values will have high entropy while still being informative about the center.

For a Beta(α, β) distribution:
- Mean: $\mu = \frac{\alpha}{\alpha + \beta}$
- To get high entropy (concave distribution), we want small α and β values (both < 1 makes it U-shaped, but values around 2-5 give high entropy while remaining unimodal)

Let's use α = 3, and solve for β to get mean = 0.17:
- $0.17 = \frac{3}{3 + \beta}$
- $\beta = \frac{3}{0.17} - 3 \approx 14.65$

After observing the data, the posterior will be Beta(xA + α, nA - xA + β)

In [ ]:
# Informative prior parameters
target_prior_mean = control_group_conversion_rate - epsilon  # 0.17
alpha_prior = 3  # Small value for high entropy
beta_prior = (alpha_prior / target_prior_mean) - alpha_prior  # Solve for beta given mean

print(f"Prior: Beta({alpha_prior:.2f}, {beta_prior:.2f})")
print(f"Prior mean: {alpha_prior / (alpha_prior + beta_prior):.4f}")
print(f"Prior variance: {(alpha_prior * beta_prior) / ((alpha_prior + beta_prior)**2 * (alpha_prior + beta_prior + 1)):.6f}")

# Posterior parameters after observing data
alpha_posterior = xA_observed + alpha_prior
beta_posterior = (nA - xA_observed) + beta_prior

print(f"\nPosterior: Beta({alpha_posterior:.2f}, {beta_posterior:.2f})")
posterior_mean = alpha_posterior / (alpha_posterior + beta_posterior)
print(f"Posterior mean: {posterior_mean:.4f}")

# Compute 95% credible interval
p_L_informative = beta_dist.ppf(0.025, alpha_posterior, beta_posterior)
p_U_informative = beta_dist.ppf(0.975, alpha_posterior, beta_posterior)
print(f"95% Credible Interval: [{p_L_informative:.4f}, {p_U_informative:.4f}]")

# Compute 99% credible interval
p_L_99_informative = beta_dist.ppf(0.005, alpha_posterior, beta_posterior)
p_U_99_informative = beta_dist.ppf(0.995, alpha_posterior, beta_posterior)
print(f"99% Credible Interval: [{p_L_99_informative:.4f}, {p_U_99_informative:.4f}]")

In [ ]:
# Visualize prior vs posterior
x_range = np.linspace(0, 0.5, 1000)
prior_pdf = beta_dist.pdf(x_range, alpha_prior, beta_prior)
posterior_pdf = beta_dist.pdf(x_range, alpha_posterior, beta_posterior)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_range, prior_pdf, 'b--', lw=2, label=f'Prior: Beta({alpha_prior:.1f}, {beta_prior:.1f})')
ax.plot(x_range, posterior_pdf, 'r-', lw=2, label=f'Posterior: Beta({alpha_posterior:.1f}, {beta_posterior:.1f})')

# Mark the non-inferiority boundary
ax.axvline(control_group_conversion_rate - epsilon, color='k', ls=':', lw=1.5, 
           label=f'Non-inferiority boundary = {control_group_conversion_rate - epsilon:.2f}')

# Mark the control conversion rate
ax.axvline(control_group_conversion_rate, color='g', ls=':', lw=1.5, 
           label=f'Control conversion rate = {control_group_conversion_rate:.2f}')

# Shade the 95% credible interval
mask = (x_range >= p_L_informative) & (x_range <= p_U_informative)
ax.fill_between(x_range[mask], posterior_pdf[mask], alpha=0.3, color='red', 
                label='95% Credible Interval')

ax.set_xlabel('Conversion rate p_A', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title('Informative Prior vs Posterior Distribution', fontsize=14)
ax.legend(loc='best')
ax.grid(True, ls=':', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Compute probabilityy that variant is non-inferior (p_A > p_C - epsilon)
# This is P(p_A > 0.17) under the posterior
prob_non_inferior = 1 - beta_dist.cdf(control_group_conversion_rate - epsilon, 
                                       alpha_posterior, beta_posterior)
print(f"Probability that variant is non-inferior: {prob_non_inferior:.4f}")
print(f"This means there's a {prob_non_inferior*100:.2f}% probabilityy that the variant conversion rate is above {control_group_conversion_rate - epsilon:.2f}")